In [ ]:
import sqlite3
import pandas as pd

# Load datasets
logon_df = pd.read_csv('logon_cleaned.csv')
email_df = pd.read_csv('email_cleaned.csv')
file_df = pd.read_csv('file_cleaned.csv')
device_df = pd.read_csv('device_cleaned.csv')

# Rename columns for consistency
logon_df.rename(columns={'user_id': 'user'}, inplace=True)

# Convert time columns to datetime format
logon_df['start_time'] = pd.to_datetime(logon_df['start_time'])
logon_df['end_time'] = pd.to_datetime(logon_df['end_time'])
email_df['date'] = pd.to_datetime(email_df['date'])
file_df['date'] = pd.to_datetime(file_df['date'])
device_df['date'] = pd.to_datetime(device_df['date'])

# Extract only the date part for proper merging
logon_df['date'] = logon_df['start_time'].dt.date
email_df['date'] = email_df['date'].dt.date
file_df['date'] = file_df['date'].dt.date
device_df['date'] = device_df['date'].dt.date

# Connect to SQLite database
db_path = "insider_threat.db"
conn = sqlite3.connect(db_path)

# Save DataFrames to SQLite
logon_df.to_sql("logon", conn, if_exists="replace", index=False)
email_df.to_sql("email", conn, if_exists="replace", index=False)
file_df.to_sql("file", conn, if_exists="replace", index=False)
device_df.to_sql("device", conn, if_exists="replace", index=False)

# SQL Query to aggregate user activities per day
query = """
    WITH Logon_Aggregate AS (
        SELECT 
            user, 
            date,
            MIN(start_time) AS first_logon_time,
            MAX(end_time) AS last_logoff_time,
            SUM((JULIANDAY(end_time) - JULIANDAY(start_time)) * 86400) AS total_working_seconds,
            SUM(logon_on_own_pc) AS total_logon_own_pc,
            SUM(logon_on_other_pc) AS total_logon_other_pc,
            SUM(logon_on_own_pc_normal) AS total_logon_own_pc_normal,
            SUM(logon_on_own_pc_off_hour) AS total_logon_own_pc_off,
            SUM(logon_on_other_pc_normal) AS total_logon_other_pc_normal,
            SUM(logon_on_other_pc_off_hour) AS total_logon_other_pc_off
        FROM logon
        GROUP BY user, date
    ),
    Email_Aggregate AS (
        SELECT 
            user, 
            date,
            SUM(int_to_int_mails + int_to_out_mails + out_to_int_mails + out_to_out_mails) AS total_emails_sent,
            SUM(after_hour_mails) AS after_hour_emails,
            SUM(internal_recipients) AS total_internal_recipients,
            SUM(external_recipients) AS total_external_recipients,
            SUM(distinct_bcc) AS total_bcc_recipients,
            SUM(mails_with_attachments) AS mails_with_attachments
        FROM email
        GROUP BY user, date
    ),
    File_Aggregate AS (
        SELECT 
            user, 
            date,
            SUM(documents_copy_own_pc) AS documents_copy_own_pc,
            SUM(documents_copy_other_pc) AS documents_copy_other_pc,
            SUM(program_files_copy_own_pc) AS program_files_copy_own_pc,
            SUM(program_files_copy_other_pc) AS program_files_copy_other_pc,
            SUM(documents_copy_own_pc_off_hour) AS documents_copy_own_pc_off_hour,
            SUM(documents_copy_other_pc_off_hour) AS documents_copy_other_pc_off_hour,
            SUM(program_files_copy_own_pc_off_hour) AS program_files_copy_own_pc_off_hour,
            SUM(program_files_copy_other_pc_off_hour) AS program_files_copy_other_pc_off_hour
        FROM file
        GROUP BY user, date
    ),
    Device_Aggregate AS (
        SELECT 
            user, 
            date,
            SUM(device_connects_on_own_pc_normal_hour) AS device_connects_on_own_pc_normal_hour,
            SUM(device_connects_on_other_pc_normal_hour) AS device_connects_on_other_pc_normal_hour,
            SUM(device_connects_on_own_pc_off_hour) AS device_connects_on_own_pc_off_hour,
            SUM(device_connects_on_other_pc_off_hour) AS device_connects_on_other_pc_off_hour
        FROM device
        GROUP BY user, date
    )
    SELECT 
        l.user, 
        l.date,
        l.first_logon_time,
        l.last_logoff_time,
        l.total_working_seconds,
        l.total_logon_own_pc,
        l.total_logon_other_pc,
        l.total_logon_own_pc_normal,
        l.total_logon_own_pc_off,
        l.total_logon_other_pc_normal,
        l.total_logon_other_pc_off,
        COALESCE(e.total_emails_sent, 0) AS total_emails_sent,
        COALESCE(e.after_hour_emails, 0) AS after_hour_emails,
        COALESCE(e.total_internal_recipients, 0) AS total_internal_recipients,
        COALESCE(e.total_external_recipients, 0) AS total_external_recipients,
        COALESCE(e.total_bcc_recipients, 0) AS total_bcc_recipients,
        COALESCE(e.mails_with_attachments, 0) AS mails_with_attachments,
        COALESCE(f.documents_copy_own_pc, 0) AS documents_copy_own_pc,
        COALESCE(f.documents_copy_other_pc, 0) AS documents_copy_other_pc,
        COALESCE(f.program_files_copy_own_pc, 0) AS program_files_copy_own_pc,
        COALESCE(f.program_files_copy_other_pc, 0) AS program_files_copy_other_pc,
        COALESCE(f.documents_copy_own_pc_off_hour, 0) AS documents_copy_own_pc_off_hour,
        COALESCE(f.documents_copy_other_pc_off_hour, 0) AS documents_copy_other_pc_off_hour,
        COALESCE(f.program_files_copy_own_pc_off_hour, 0) AS program_files_copy_own_pc_off_hour,
        COALESCE(f.program_files_copy_other_pc_off_hour, 0) AS program_files_copy_other_pc_off_hour,
        COALESCE(d.device_connects_on_own_pc_normal_hour, 0) AS device_connects_on_own_pc_normal_hour,
        COALESCE(d.device_connects_on_other_pc_normal_hour, 0) AS device_connects_on_other_pc_normal_hour,
        COALESCE(d.device_connects_on_own_pc_off_hour, 0) AS device_connects_on_own_pc_off_hour,
        COALESCE(d.device_connects_on_other_pc_off_hour, 0) AS device_connects_on_other_pc_off_hour
    FROM Logon_Aggregate l
    LEFT JOIN Email_Aggregate e ON l.user = e.user AND l.date = e.date
    LEFT JOIN File_Aggregate f ON l.user = f.user AND l.date = f.date
    LEFT JOIN Device_Aggregate d ON l.user = d.user AND l.date = d.date;
"""

# Execute query and save results
feature_vector = pd.read_sql_query(query, conn)

# Save to CSV
feature_vector.to_csv("feature_vector.csv", index=False)

print("Feature vector generated and saved as 'feature_vector.csv' successfully!")

# Close connection
conn.close()


In [49]:
import pandas as pd
data=pd.read_csv('feature_vector.csv')

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
data['user'].nunique()

In [ ]:
data.columns

In [ ]:
data['device_connects_on_other_pc_off_hour'].value_counts()

In [56]:
threat=pd.read_csv('insiders.csv')

In [ ]:
threat.head()


In [58]:
threat.drop(columns=['dataset','details','end'],inplace=True)

In [ ]:
threat.head()

In [17]:
import pandas as pd

# Load CSV files into Pandas DataFrames
feature_vector_df = pd.read_csv("feature_vector.csv")
insiders_df = pd.read_csv("/Users/truptijagdale/Desktop/tensorflow_env/anomaly (1).csv")
insiders_df.rename(columns={"start": "first_logon_time", "end": "last_logoff_time",'user_id_decoded':'user'}, inplace=True)
# Ensure timestamps are in datetime format
feature_vector_df["first_logon_time"] = pd.to_datetime(feature_vector_df["first_logon_time"])
feature_vector_df["last_logoff_time"] = pd.to_datetime(feature_vector_df["last_logoff_time"])
insiders_df["first_logon_time"] = pd.to_datetime(insiders_df["first_logon_time"])

/var/folders/x6/927q59w17dg3mzfmhv4v0g6r0000gn/T/ipykernel_2525/3980601641.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  insiders_df["first_logon_time"] = pd.to_datetime(insiders_df["first_logon_time"])


In [18]:
insiders_df.columns

Index(['user', 'first_logon_time', 'last_logoff_time'], dtype='object')

In [22]:
insiders_df.isnull().sum()

user                0
first_logon_time    0
last_logoff_time    0
dtype: int64

In [21]:
insiders_df.dropna(inplace=True)

In [23]:

# Initialize 'is_anomaly' column with 0 (default: not an anomaly)
feature_vector_df["is_anomaly"] = 0

# Iterate over each row in insiders_df and mark anomalies in feature_vector_df
for _, insider in insiders_df.iterrows():
    mask = (
        (feature_vector_df["user"] == insider["user"]) &
        (feature_vector_df["first_logon_time"] <= insider["first_logon_time"]) &
        (feature_vector_df["last_logoff_time"] >= insider["first_logon_time"])
    )
    feature_vector_df.loc[mask, "is_anomaly"] = 1  # Mark as anomaly

# Save the updated DataFrame with the 'is_anomaly' column
feature_vector_df.to_csv("merged_data.csv", index=False)

# Display first 10 rows
print(feature_vector_df.head(10))


      user        date    first_logon_time    last_logoff_time  \
0  AAE0190  2010-01-04 2010-01-04 08:09:00 2010-01-04 18:20:00   
1  AAE0190  2010-01-05 2010-01-05 08:19:00 2010-01-05 18:11:00   
2  AAE0190  2010-01-06 2010-01-06 08:09:00 2010-01-06 18:10:00   
3  AAE0190  2010-01-07 2010-01-07 08:23:00 2010-01-07 18:14:00   
4  AAE0190  2010-01-08 2010-01-08 08:17:00 2010-01-08 18:26:00   
5  AAE0190  2010-01-11 2010-01-11 08:09:00 2010-01-11 18:24:00   
6  AAE0190  2010-01-12 2010-01-12 08:12:00 2010-01-12 18:30:00   
7  AAE0190  2010-01-13 2010-01-13 08:15:00 2010-01-13 18:14:00   
8  AAE0190  2010-01-14 2010-01-14 08:14:00 2010-01-14 18:29:00   
9  AAE0190  2010-01-15 2010-01-15 08:19:00 2010-01-15 18:13:00   

   total_working_seconds  total_logon_own_pc  total_logon_other_pc  \
0           36660.000028                   1                     0   
1           35520.000015                   1                     0   
2           36060.000023                   1                   

In [24]:
feature_vector_df.columns

Index(['user', 'date', 'first_logon_time', 'last_logoff_time',
       'total_working_seconds', 'total_logon_own_pc', 'total_logon_other_pc',
       'total_logon_own_pc_normal', 'total_logon_own_pc_off',
       'total_logon_other_pc_normal', 'total_logon_other_pc_off',
       'total_emails_sent', 'after_hour_emails', 'total_internal_recipients',
       'total_external_recipients', 'total_bcc_recipients',
       'mails_with_attachments', 'documents_copy_own_pc',
       'documents_copy_other_pc', 'program_files_copy_own_pc',
       'program_files_copy_other_pc', 'documents_copy_own_pc_off_hour',
       'documents_copy_other_pc_off_hour',
       'program_files_copy_own_pc_off_hour',
       'program_files_copy_other_pc_off_hour',
       'device_connects_on_own_pc_normal_hour',
       'device_connects_on_other_pc_normal_hour',
       'device_connects_on_own_pc_off_hour',
       'device_connects_on_other_pc_off_hour', 'is_anomaly'],
      dtype='object')

In [25]:
feature_vector_df['is_anomaly'].value_counts()

is_anomaly
0    307753
1     22515
Name: count, dtype: int64

In [26]:
feature_vector_df.isnull().sum()

user                                       0
date                                       0
first_logon_time                           0
last_logoff_time                           0
total_working_seconds                      0
total_logon_own_pc                         0
total_logon_other_pc                       0
total_logon_own_pc_normal                  0
total_logon_own_pc_off                     0
total_logon_other_pc_normal                0
total_logon_other_pc_off                   0
total_emails_sent                          0
after_hour_emails                          0
total_internal_recipients                  0
total_external_recipients                  0
total_bcc_recipients                       0
mails_with_attachments                     0
documents_copy_own_pc                      0
documents_copy_other_pc                    0
program_files_copy_own_pc                  0
program_files_copy_other_pc                0
documents_copy_own_pc_off_hour             0
documents_

In [27]:
feature_vector_df.shape

(330268, 30)

In [28]:
feature_vector_df.drop_duplicates(inplace=True)

In [29]:
feature_vector_df.shape

(330268, 30)

In [30]:
feature_vector_df.to_csv('final_feature_vector.csv',index=False)